In [3]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from mne.preprocessing import ICA,read_ica

In [4]:
#read the ica file from the folder
sample_data_raw_file=(r'C:\Users\user\Desktop\Winter data\Winter ICA\19122019_2.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw_temp=raw.copy()#.crop(600., 620.) #Copy the raw data and crop the 30 sec data basically skype call
raw_ch = raw_temp.pick_channels(['AF3', 'F3', 'F4', 'AF4']) # select only one channel

Opening raw data file C:\Users\user\Desktop\Winter data\Winter ICA\19122019_2.fif...
    Range : 0 ... 361215 =      0.000 ...  1410.996 secs
Ready.
Reading 0 ... 361215  =      0.000 ...  1410.996 secs...


<ipython-input-4-f526042a28a8>:3: RuntimeWarning: This filename (C:\Users\user\Desktop\Winter data\Winter ICA\19122019_2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)


In [5]:
raw_temp.info['ch_names']

['AF3', 'F3', 'F4', 'AF4']

In [6]:
ey_op =raw_ch.copy().crop(6., 16)# copy the data
ey_cl =raw_ch.copy().crop(27., 37.)
phase_1 =raw_ch.copy().crop(389., 399.)
phase_2 =raw_ch.copy().crop(811., 821.)
#m2_bf =raw_ch.copy().crop(1250., 1260.)
#m2_af =raw_ch.copy().crop(1277., 1287.)
#ph_4_bf =raw_ch.copy().crop(1605., 1615.)
#ph_4_af =raw_ch.copy().crop(1655., 1665.)



#data,times = sk_bf[:]
#print(data.size)
#print(data)
#sk_bf.plot(scalings={"eeg": 75e-6})

In [7]:
#data0,times0 = ph0[:] #extract times and data
#print(len(data))
#print(data.shape)
#s=1/times
#print(len(s))
#plt.plot(times, data.T)
#plt.show()
#data_sk,times_sk= ph1_sk[:] 
#data_m1,times_m1= ph2_m1[:]
#data_m2,times_m2= ph2_m2[:]
#data_3,times_3= ph_3[:]
#data_4,times_4= ph_4[:]
#data_5,times_5= ph_5[:]

In [8]:
from mne.time_frequency import psd_welch
def eeg_power_band(data):
    """EEG power band feature extraction.


    """
    # specific frequency bands
    FREQ_BANDS = {#"delta": [0.5, 4.5],
                  #"theta": [4, 8],
                  "alpha": [8, 13],
                  "beta": [13, 30]}
                  #"gamma": [30, 56]}

    psds, freqs = psd_welch(data, picks=['AF3', 'F3', 
                                          'F4', 'AF4'],
                            fmin=8, fmax=30.)
    
    #print(psds.shape)
    #print(freqs.shape)
    #plt.semilogy(freqs, (psds.T))
    #plt.xlabel('frequency [Hz]')
    #plt.ylabel('Linear spectrum [V RMS]')
    #plt.show()
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:,  (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)    #:,
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [9]:
power_ey_op = eeg_power_band(ey_op)
#print(power_ey_op)
power_ey_cl = eeg_power_band(ey_cl)
#print(power_ey_cl)
power_1 = eeg_power_band(phase_1)
power_2 = eeg_power_band(phase_2)
#power_4 = eeg_power_band(m2_bf)
#power_5 = eeg_power_band(m2_af)
#power_6 = eeg_power_band(ph_4_bf)
#power_7 = eeg_power_band(ph_4_af)

Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)


In [10]:
import pandas as pd
ch_name = ['AF3', 'F3',  'F4', 'AF4']
column = ["alpha","beta"]

#print(F1)


In [11]:
df_0= pd.DataFrame(power_ey_op, columns = column)
F0 = df_0.assign(**{'ch_name':ch_name,
               'phase': 'eye_op'})
F0
df_1= pd.DataFrame(power_ey_cl, columns = column)
F1 = df_1.assign(**{'ch_name':ch_name,
               'phase':'eye_cl'})
df_2= pd.DataFrame(power_1, columns = column)
F2 = df_2.assign(**{'ch_name':ch_name,
               'phase': 1})
df_3= pd.DataFrame(power_2, columns = column)
F3 = df_3.assign(**{'ch_name':ch_name,
               'phase': 2})


In [12]:
result =result = pd.concat([F0,F1,F2,F3], sort=False)
result.set_index('phase',inplace=True)
result

,alpha,beta,ch_name
phase,,,
eye_op,0.058668,0.038422,AF3
eye_op,0.074270,0.035793,F3
eye_op,0.088396,0.031478,F4
eye_op,0.070029,0.035174,AF4
eye_cl,0.063516,0.038127,AF3
eye_cl,0.088172,0.031239,F3
eye_cl,0.071012,0.035423,F4
eye_cl,0.068492,0.036351,AF4
1,0.038381,0.044395,AF3


In [13]:
#save the data to a csv_file.
result.to_csv(r'C:\Users\user\Desktop\Winter data\Winter ICA\feature\19122019_2.csv')